In [2]:
from pathlib import Path
import requests

# List of PMC article URLs
urls = [
    "https://pmc.ncbi.nlm.nih.gov/articles/PMC4863427",
    "https://pmc.ncbi.nlm.nih.gov/articles/PMC7223160",
    "https://pmc.ncbi.nlm.nih.gov/articles/PMC7264388",
]

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/125.0.0.0 Safari/537.36"
    ),
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://pmc.ncbi.nlm.nih.gov/",
}

for url in urls:
    pmcid = url.split("/")[-1]
    outfile = Path(f"{pmcid}.html")
    with requests.Session() as s:
        s.headers.update(headers)
        resp = s.get(url, timeout=30)
        resp.raise_for_status()
        if not resp.encoding:
            resp.encoding = "utf-8"
        outfile.write_text(resp.text, encoding=resp.encoding)
    print(f"Saved {outfile.resolve()} ({outfile.stat().st_size:,} bytes)")

Saved C:\Users\USER\github-classroom\ntu-info\neurosynth-etl-11rachelh\PMC4863427.html (180,248 bytes)
Saved C:\Users\USER\github-classroom\ntu-info\neurosynth-etl-11rachelh\PMC7223160.html (357,542 bytes)
Saved C:\Users\USER\github-classroom\ntu-info\neurosynth-etl-11rachelh\PMC7223160.html (357,542 bytes)
Saved C:\Users\USER\github-classroom\ntu-info\neurosynth-etl-11rachelh\PMC7264388.html (205,454 bytes)
Saved C:\Users\USER\github-classroom\ntu-info\neurosynth-etl-11rachelh\PMC7264388.html (205,454 bytes)


In [3]:
# Extraction handles multi-row headers (e.g., 'Coordinates' with x, y, z underneath), flattens headers, and extracts correct columns, including negative values
import re
import pandas as pd
from bs4 import BeautifulSoup
from pathlib import Path

html_files = [
    'PMC4863427.html',
    'PMC7223160.html',
    'PMC7264388.html',
]

results = []

def extract_keywords(soup, html_text):
    meta_keywords = soup.find('meta', attrs={'name': 'citation_keywords'})
    if meta_keywords and meta_keywords.get('content'):
        return meta_keywords.get('content')
    for tag in soup.find_all(['b', 'strong', 'h3', 'h4', 'p', 'span']):
        if tag.get_text(strip=True).lower().startswith('keywords'):
            next_text = ''
            text = tag.get_text(separator=' ', strip=True)
            if ':' in text:
                next_text = text.split(':', 1)[1].strip()
            if not next_text:
                sibling = tag.find_next_sibling(text=True)
                if sibling:
                    next_text = sibling.strip()
            if not next_text and tag.parent:
                sibling = tag.parent.find_next_sibling(text=True)
                if sibling:
                    next_text = sibling.strip()
            if next_text:
                return next_text
    kw_match = re.search(r'Keywords?[:\s]+([\w\s;,-]+)', html_text, re.IGNORECASE)
    if kw_match:
        return kw_match.group(1).strip()
    return ''

def is_number(val):
    try:
        # Remove spaces between minus and number, e.g., '- 36' -> '-36'
        val = re.sub(r'^-\s+', '-', val.strip())
        float(val)
        return True
    except Exception:
        return False

def find_xyz_indices_from_multiline_header(header_rows):
    n_cols = max(len(row) for row in header_rows)
    flat_headers = [''] * n_cols
    for row in header_rows:
        for i in range(n_cols):
            part = row[i].strip().lower() if i < len(row) else ''
            if part:
                if flat_headers[i]:
                    flat_headers[i] += ' ' + part
                else:
                    flat_headers[i] = part
    x_idx = y_idx = z_idx = None
    for i, h in enumerate(flat_headers):
        if x_idx is None and re.search(r'\bx\b', h):
            x_idx = i
        elif y_idx is None and re.search(r'\by\b', h):
            y_idx = i
        elif z_idx is None and re.search(r'\bz\b', h):
            z_idx = i
    if None not in (x_idx, y_idx, z_idx):
        return x_idx, y_idx, z_idx
    return None

for html_file in html_files:
    html_path = Path(html_file)
    if not html_path.exists():
        print(f"File not found: {html_file}")
        continue
    html_text = html_path.read_text(encoding='utf-8', errors='ignore')
    soup = BeautifulSoup(html_text, 'html.parser')

    pmcid = re.search(r'PMC\d+', html_file).group() if re.search(r'PMC\d+', html_file) else ''

    pmid = ''
    meta_pmid = soup.find('meta', attrs={'name': 'citation_pmid'})
    if meta_pmid:
        pmid = meta_pmid.get('content')
    else:
        pmid_match = re.search(r'PMID[:\s]+(\d+)', html_text)
        if pmid_match:
            pmid = pmid_match.group(1)

    keywords = extract_keywords(soup, html_text)

    tables = soup.find_all('table')
    for table_idx, table in enumerate(tables, 1):
        rows = table.find_all('tr')
        if not rows:
            continue
        header_rows = []
        for row in rows[:2]:
            cells = row.find_all(['th', 'td'])
            if all(cell.name == 'th' for cell in cells):
                header_rows.append([cell.get_text(strip=True) for cell in cells])
        if not header_rows:
            header_rows = [[cell.get_text(strip=True) for cell in rows[0].find_all(['th', 'td'])]]
        xyz_indices = find_xyz_indices_from_multiline_header(header_rows)
        if xyz_indices:
            x_idx, y_idx, z_idx = xyz_indices
            for row in rows[len(header_rows):]:
                cols = [td.get_text(strip=True) for td in row.find_all(['td', 'th'])]
                if len(cols) > max(x_idx, y_idx, z_idx):
                    x_val, y_val, z_val = cols[x_idx], cols[y_idx], cols[z_idx]
                    # Remove spaces between minus and number for each value
                    x_val = re.sub(r'^-\s+', '-', x_val.strip())
                    y_val = re.sub(r'^-\s+', '-', y_val.strip())
                    z_val = re.sub(r'^-\s+', '-', z_val.strip())
                    if all(is_number(v) for v in [x_val, y_val, z_val]):
                        x = float(x_val)
                        y = float(y_val)
                        z = float(z_val)
                        results.append({
                            'PMID': pmid,
                            'PMCID': pmcid.replace('PMC',''),
                            'Keywords': keywords,
                            'Table': table_idx,
                            'X': x,
                            'Y': y,
                            'Z': z,
                        })

df = pd.DataFrame(results)
df.to_csv('b11504006_info_data.csv', index=False)
print('Saved to b11504006_info_data.csv')

Saved to b11504006_info_data.csv
